# Running MATLAB from PYTHON 

In [1]:
import numpy as np

import os
import subprocess as sp
import shlex

## 1. Very Simple Commands, use a !

In [2]:
# simple commands can be run with a !
!ls

CT_out		     mw_gsw_CT_from_t.m~  mywrapper.m	temporaryfile
foo.sh		     mw_gsw_SA_from_SP.m  mywrapper.m~	Untitled.ipynb
matfiles	     myoutput		  results.txt	wrapper_gsw_check.m
matfiles2	     myoutputfile	  SA_out	wrapper_gsw_check.m~
MatlabScripts.ipynb  myscript.m		  startup.m
mw_gsw_CT_from_t.m   myscript.m~	  stuff


## 2. OS Provides a Number of Useful Commands

In [4]:
# more complicated commands, say where you want to use a variable file name,  can be run with os
newdirectory = 'matfiles'
os.mkdir(newdirectory)
!ls -F

CT_out		     mw_gsw_CT_from_t.m~  mywrapper.m	temporaryfile
foo.sh		     mw_gsw_SA_from_SP.m  mywrapper.m~	Untitled.ipynb
matfiles/	     myoutput		  results.txt	wrapper_gsw_check.m
matfiles2/	     myoutputfile	  SA_out	wrapper_gsw_check.m~
MatlabScripts.ipynb  myscript.m		  startup.m
mw_gsw_CT_from_t.m   myscript.m~	  stuff


## 3. To Call Scripts use Subprocess, each piece in ""

In [5]:
# and calling scripts
!cat myscript.m

function [] = myscript(filename, x)
  y = sin(x);
  dlmwrite(filename, y, ' ')
end





In [6]:
x = np.arange(0, 1, 0.1)
filename = "'myoutput'"
functioncall = 'myscript(' + filename + ', '+ str(x) + ');exit'
print (functioncall)
cmd = ["matlab","-nodesktop","-nodisplay","-r", functioncall]
print (cmd)
sp.run(cmd)
!cat myoutput

myscript('myoutput', [ 0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9]);exit
['matlab', '-nodesktop', '-nodisplay', '-r', "myscript('myoutput', [ 0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9]);exit"]
0 0.099833 0.19867 0.29552 0.38942 0.47943 0.56464 0.64422 0.71736 0.78333


But this super messy.  Lets make it a function.

In [7]:
def make_cmd(scriptname, filename, args):

    singlequote = "'"
    comma = ','
    
    # add quotes to the filename
    mfilename = '{0}{1}{0}'.format(singlequote, filename)
    
    firstpart = '{0}('.format(scriptname)
    lastpart = ');exit'
    
    functioncall = firstpart + mfilename
    for arg in args:
        functioncall += comma + str(arg)
    functioncall += lastpart
    
    cmd = shlex.split('matlab -nodesktop -nodisplay -r')
    cmd.append(functioncall)
    return cmd

In [8]:
cmd = make_cmd('myscript', 'myoutput', [x])
sp.run(cmd)

CompletedProcess(args=['matlab', '-nodesktop', '-nodisplay', '-r', "myscript('myoutput',[ 0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9]);exit"], returncode=0)

# 4. GSW

Okay, now we are set to use the MATLAB toolbox gsw.  Note info on this toolbox is here: http://www.teos-10.org/pubs/Getting_Started.pdf

In [9]:
# Some data
long = -123
lat = 49
SP = [ 34.5759,  34.2870,  34.5888,  34.6589,  34.6798,  34.6910,  34.6956 ] 
t  = [ 19.5076,   3.6302,   1.9533,   1.5661,   1.4848,   1.4989,   1.5919 ] 
p  = [       0,     1010,     2025,     3045,     4069,     5098,     6131 ] 

In [10]:
# our matlab wrapper script
!cat mw_gsw_SA_from_SP.m

function [] = mw_gsw_SA_from_SP(filename, SP, p, long, lat)
  startup()
  y = gsw_SA_from_SP(SP, p, long, lat)
  dlmwrite(filename, y, ',')


In [11]:
# our start up script (needs to point to where you installed GSW)
!cat startup.m

function [] = startup()
  addpath /ocean/rich/home/matlab/gsw3
  addpath /ocean/rich/home/matlab/gsw3/html
  addpath /ocean/rich/home/matlab/gsw3/library
  addpath /ocean/rich/home/matlab/gsw3/thermodynamics_from_t
  addpath /ocean/rich/home/matlab/gsw3/pdf


In [12]:
filename = 'SA_out'
cmd = make_cmd('mw_gsw_SA_from_SP', filename, [SP, p, long, lat])
sp.run(cmd)

# and get the data out of the file
with open(filename, 'r') as fn:
    SAs = fn.readline().split(',')
    SA = [float(S) for S in SAs]

Lets wrap that in a function

In [13]:
def run_matlab(scriptname, args):
    filename = 'temporaryfile'
    cmd = make_cmd(scriptname, filename, args)
    sp.run(cmd)
    with open(filename, 'r') as fn:
        strings = fn.readline().split(',')
        floats = [float(string) for string in strings]
    return floats

## and now use it to calculate conservative temperature

In [14]:
# our wrapper, note how similar it is to the previous one
!cat mw_gsw_CT_from_t.m

function [] = mw_gsw_CT_from_t(filename, SA, t, p)
  startup()
  y = gsw_CT_from_t(SA, t, p)
  dlmwrite(filename, y, ',')


In [15]:
CT = run_matlab('mw_gsw_CT_from_t', [SA, t, p])
print (CT)

[19.513, 3.5587, 1.8157, 1.3434, 1.1583, 1.0517, 1.0072]
